# Wrangle and analyze Data

### Project Details###

Your tasks in this project are as follows:

1. Data wrangling, which consists of:
    - Gathering data
    - Assessing data
    - Cleaning data

2. Storing, analyzing, and visualizing your wrangled data
3. Reporting on:
    A. your data wrangling efforts and 
    B. your data analyses and visualizations

### Gathering Data for this Project###

Gather each of the three pieces of data as described below in a Jupyter Notebook titled `wrangle_act.ipynb`:

1. The WeRateDogs Twitter archive. I am giving this file to you, so imagine it as a file on hand. Download this file manually by clicking the following link: [twitter_archive_enhanced.csv](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv)

2. The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (`image_predictions.tsv`) is hosted on Udacity's servers and should be downloaded programmatically using the [Requests](http://docs.python-requests.org/en/master/) library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

3. Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the *tweet IDs* in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's [Tweepy](http://www.tweepy.org/) library and store each tweet's entire set of JSON data in a file called **`tweet_json.txt`** file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum):
    - tweet ID 
    - retweet count 
    - favorite count 

*Note: do not include your Twitter API keys, secrets, and tokens in your project submission.*

If you decide to complete your project in the Project Workspace, note that you can upload files to the Jupyter Notebook Workspace by clicking the "Upload" button in the top righthand corner of the dashboard.

In [31]:
import numpy as np
import pandas as pd
import json
import requests
import time
import tweepy

In [2]:
# load the data part 1 of 3

archive_df = pd.read_csv('./twitter-archive-enhanced.csv', encoding = 'utf-8')

archive_df.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

**Missing Data**
- `in_reply_to_status_id`: 2278 missing values.
- `in_reply_to_user_id`: 2278 missing values.
- `retweeted_status_id`: 2175 missing values.
- `retweeted_status_user_id`: 2175 missing values.
- `retweeted_status_timestamp`: 2175 missing values.
- `expanded_urls`: 59 missing values.

In [4]:
# load the data part 2 of 3

# use requests library to download the image prediction file using the link provided in the documentation.
predct_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(predct_url, auth=('user', 'pass'))

# save the downloaded file 
with open(predct_url.split('/')[-1], mode = 'wb') as outfile:
    outfile.write(response.content)

# load the file into a dataframe
img_predict_df = pd.read_csv('image-predictions.tsv', sep = '\t', encoding = 'utf-8')

img_predict_df.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [5]:
img_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


The image prediction file has 2075 rows and 12 columns.

In [6]:
# load the data part 3 of 3: Tweeter API

CONSUMER_KEY = "iLdhdumhWW1Y6y4ORMV2v7tba"
CONSUMER_SECRET = "9HdBKdp2F95rcFHyURFtS1R43D9WGYaQN0LuJAhK6BmJN62588"
ACCESS_TOKEN = "994807626952306688-1U0Zgng0scxbKR8xYJp5GrnNO5h1tqM"
ACCESS_TOKEN_SECRET = "nFgomxmaB60Vo3mkC0SwHmA7u2AXK37YU7I5oBlcuFN5I"

# Create an OAuthHandler instance

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth, 
                 parser = tweepy.parsers.JSONParser(), 
                 wait_on_rate_limit = True,
                 wait_on_rate_limit_notify = True)

In [17]:
# Create a list to save the tweet dictionaries returned by the API
tweets_list = []
# Create a list to save the tweet_ids that are not found by the API
missing_tweets_list = []

# Calculate the time of excution: TBD
start = time.time()

# Get the tweet object for all the teweets in archive dataframe 

for tweet_id in archive_df['tweet_id']:
    try:
        tweet_status = api.get_status(tweet_id, tweet_mode = 'extended')
        # Print one page to look at the structure of the returned file
        # and the names of attributes
        # print(json.dumps(page, indent = 4))
        #break
        
        retweet_count = tweet_status['retweet_count'] 
        favorite_count = tweet_status['favorite_count'] 
        user_followers = tweet_status['user']['followers_count'] 
        user_favourites = tweet_status['user']['favourites_count']
        date_time = tweet_status['created_at'] 
        
        tweets_list.append({'tweet_id': int(tweet_id),
                        'retweet_count': int(retweet_count),
                        'favorite_count': int(favorite_count),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
    
    # Catch the exceptions of the TweepError
    except Exception as exp:
        print(str(tweet_id)+ " - " + str(exp))
        missing_tweets_list.append(tweet_id)

# exceution time:
end = time.time()
print(end - start)

888202515573088257 _ [{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921 _ [{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329 _ [{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744 _ [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320 _ [{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705 _ [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648 _ [{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308 _ [{'code': 144, 'message': 'No status found with that ID.'}]
827228250799742977 _ [{'code': 144, 'message': 'No status found with that ID.'}]
802247111496568832 _ [{'code': 144, 'message': 'No status found with that ID.'}]
775096608509886464 _ [{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 696


758740312047005698 _ Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))


Rate limit reached. Sleeping for: 698


676957860086095872 _ Failed to send request: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')",))
1974.1083195209503


In [22]:
# the number of returned and missing tweets

len(tweets_list),len(missing_tweets_list)

(2343, 13)

Notes:
- As we can see above, the tweepy rate limit was reached twice.
- 

In [23]:
missing_tweets_list

[888202515573088257,
 873697596434513921,
 869988702071779329,
 866816280283807744,
 861769973181624320,
 845459076796616705,
 842892208864923648,
 837012587749474308,
 827228250799742977,
 802247111496568832,
 775096608509886464,
 758740312047005698,
 676957860086095872]

In [24]:
tweets_list

[{'date_time': Timestamp('2017-08-01 16:23:56'),
  'favorite_count': 38859,
  'retweet_count': 8610,
  'tweet_id': 892420643555336193,
  'user_favourites': 133155,
  'user_followers': 6783322},
 {'date_time': Timestamp('2017-08-01 00:17:27'),
  'favorite_count': 33283,
  'retweet_count': 6324,
  'tweet_id': 892177421306343426,
  'user_favourites': 133155,
  'user_followers': 6783324},
 {'date_time': Timestamp('2017-07-31 00:18:03'),
  'favorite_count': 25075,
  'retweet_count': 4195,
  'tweet_id': 891815181378084864,
  'user_favourites': 133155,
  'user_followers': 6783324},
 {'date_time': Timestamp('2017-07-30 15:58:51'),
  'favorite_count': 42236,
  'retweet_count': 8722,
  'tweet_id': 891689557279858688,
  'user_favourites': 133155,
  'user_followers': 6783324},
 {'date_time': Timestamp('2017-07-29 16:00:24'),
  'favorite_count': 40382,
  'retweet_count': 9491,
  'tweet_id': 891327558926688256,
  'user_favourites': 133155,
  'user_followers': 6783324},
 {'date_time': Timestamp('2017

In [ ]:
# We repeat the same operation for the tweet_ids that we coudln't get and append the result to df_list
missing_tweets_rerun_list = []
for twt in missing_tweets_list:
    try:
        favorites = page['favorite_count']
        retweets = page['retweet_count']
        user_followers = page['user']['followers_count']
        user_favourites = page['user']['favourites_count']
        date_time = page['created_at']
        
        df_list.append({'tweet_id': int(tweet_id),
                        'favorites': int(favorites),
                        'retweets': int(retweets),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
        
    except Exception:
        print(str(tweet_id)+ " _ " + str(e))
        ee_list.append(e)

In [25]:
tweet_status = api.get_status("888202515573088257", tweet_mode = 'extended')


TweepError: [{'code': 144, 'message': 'No status found with that ID.'}]

In [ ]:
missing_tweets_rerun_list = []
for twt in missing_tweets_list:
    try:
        tweet_status = api.get_status(tweet_id, tweet_mode = 'extended')
        # Print one page to look at the structure of the returned file
        # and the names of attributes
        # print(json.dumps(page, indent = 4))
        #break
        
        retweet_count = tweet_status['retweet_count'] 
        favorite_count = tweet_status['favorite_count'] 
        user_followers = tweet_status['user']['followers_count'] 
        user_favourites = tweet_status['user']['favourites_count']
        date_time = tweet_status['created_at'] 
        
        tweets_list.append({'tweet_id': int(tweet_id),
                        'retweet_count': int(retweet_count),
                        'favorite_count': int(favorite_count),
                        'user_followers': int(user_followers),
                        'user_favourites': int(user_favourites),
                        'date_time': pd.to_datetime(date_time)})
    
    # Catch the exceptions of the TweepError
    except Exception as exp:
        print(str(tweet_id)+ " - " + str(exp))
        missing_tweets_list.append(tweet_id)

# exceution time:
end = time.time()
print(end - start)

In [34]:
tweet_status = api.get_status("891327558926688256", tweet_mode = 'extended')
type(tweet_status)

dict

In [30]:
# test_dict = {}
# test_dict["891327558926688256"] = tweet_status._json

AttributeError: 'dict' object has no attribute '_json'

In [ ]:
tweet_data = {}
for tweet in archive_df['tweet_id']:
    try:
        tweet_status = api.get_status(tweet,
                                      wait_on_rate_limit=True, 
                                      wait_on_rate_limit_notify=True)
        tweet_data[str(tweet)] = tweet_status._json
    except: 
        print("Error for: " + str(tweet))

Error for: 892420643555336193
Error for: 892177421306343426
Error for: 891815181378084864
Error for: 891689557279858688
Error for: 891327558926688256
Error for: 891087950875897856
Error for: 890971913173991426
Error for: 890729181411237888
Error for: 890609185150312448
Error for: 890240255349198849
Error for: 890006608113172480
Error for: 889880896479866881
Error for: 889665388333682689
Error for: 889638837579907072
Error for: 889531135344209921
Error for: 889278841981685760
Error for: 888917238123831296
Error for: 888804989199671297
Error for: 888554962724278272
Error for: 888202515573088257
Error for: 888078434458587136
Error for: 887705289381826560
Error for: 887517139158093824
Error for: 887473957103951883
Error for: 887343217045368832
Error for: 887101392804085760
Error for: 886983233522544640
Error for: 886736880519319552
Error for: 886680336477933568
Error for: 886366144734445568
Error for: 886267009285017600
Error for: 886258384151887873
Error for: 886054160059072513
Error for:

Error for: 840698636975636481
Error for: 840696689258311684
Error for: 840632337062862849
Error for: 840370681858686976
Error for: 840268004936019968
Error for: 839990271299457024
Error for: 839549326359670784
Error for: 839290600511926273
Error for: 839239871831150596
Error for: 838952994649550848
Error for: 838921590096166913
Error for: 838916489579200512
Error for: 838831947270979586
Error for: 838561493054533637
Error for: 838476387338051585
Error for: 838201503651401729
Error for: 838150277551247360
Error for: 838085839343206401
Error for: 838083903487373313
Error for: 837820167694528512
Error for: 837482249356513284
Error for: 837471256429613056
Error for: 837366284874571778
Error for: 837110210464448512
Error for: 837012587749474308
Error for: 836989968035819520
Error for: 836753516572119041
Error for: 836677758902222849
Error for: 836648853927522308
Error for: 836397794269200385
Error for: 836380477523124226
Error for: 836260088725786625
Error for: 836001077879255040
Error for:

Error for: 805487436403003392
Error for: 805207613751304193
Error for: 804738756058218496
Error for: 804475857670639616
Error for: 804413760345620481
Error for: 804026241225523202
Error for: 803773340896923648
Error for: 803692223237865472
Error for: 803638050916102144
Error for: 803380650405482500
Error for: 803321560782307329
Error for: 803276597545603072
Error for: 802952499103731712
Error for: 802624713319034886
Error for: 802600418706604034
Error for: 802572683846291456
Error for: 802323869084381190
Error for: 802265048156610565
Error for: 802247111496568832
Error for: 802239329049477120
Error for: 802185808107208704
Error for: 801958328846974976
Error for: 801854953262350336
Error for: 801538201127157760
Error for: 801285448605831168
Error for: 801167903437357056
Error for: 801127390143516673
Error for: 801115127852503040
Error for: 800859414831898624
Error for: 800855607700029440
Error for: 800751577355128832
Error for: 800513324630806528
Error for: 800459316964663297
Error for:

Error for: 770093767776997377
Error for: 770069151037685760
Error for: 769940425801170949
Error for: 769695466921623552
Error for: 769335591808995329
Error for: 769212283578875904
Error for: 768970937022709760
Error for: 768909767477751808
Error for: 768855141948723200
Error for: 768609597686943744
Error for: 768596291618299904
Error for: 768554158521745409
Error for: 768473857036525572
Error for: 768193404517830656
Error for: 767884188863397888
Error for: 767754930266464257
Error for: 767500508068192258
Error for: 767191397493538821
Error for: 767122157629476866
Error for: 766864461642756096
Error for: 766793450729734144
Error for: 766714921925144576
Error for: 766693177336135680
Error for: 766423258543644672
Error for: 766313316352462849
Error for: 766078092750233600
Error for: 766069199026450432
Error for: 766008592277377025
Error for: 765719909049503744
Error for: 765669560888528897
Error for: 765395769549590528
Error for: 765371061932261376
Error for: 765222098633691136
Error for:

Rate limit reached. Sleeping for: 190


...

Error for: 720775346191278080
Error for: 720415127506415616
Error for: 720389942216527872
Error for: 720340705894408192
Error for: 720059472081784833
Error for: 720043174954147842
Error for: 719991154352222208
Error for: 719704490224398336
Error for: 719551379208073216
Error for: 719367763014393856
Error for: 719339463458033665
Error for: 719332531645071360
Error for: 718971898235854848
Error for: 718939241951195136
Error for: 718631497683582976
Error for: 718613305783398402
Error for: 718540630683709445
Error for: 718460005985447936
Error for: 718454725339934721
Error for: 718246886998687744
Error for: 718234618122661888
Error for: 717841801130979328
Error for: 717790033953034240
Error for: 717537687239008257
Error for: 717428917016076293
Error for: 717421804990701568
Error for: 717047459982213120
Error for: 717009362452090881
Error for: 716802964044845056
Error for: 716791146589110272
Error for: 716730379797970944
Error for: 716447146686459905
Error for: 716439118184652801
Error for:

Error for: 696713835009417216
Error for: 696518437233913856
Error for: 696490539101908992
Error for: 696488710901260288
Error for: 696405997980676096
Error for: 696100768806522880
Error for: 695816827381944320
Error for: 695794761660297217
Error for: 695767669421768709
Error for: 695629776980148225
Error for: 695446424020918272
Error for: 695409464418041856
Error for: 695314793360662529
Error for: 695095422348574720
Error for: 695074328191332352
Error for: 695064344191721472
Error for: 695051054296211456
Error for: 694925794720792577
Error for: 694905863685980160
Error for: 694669722378485760
Error for: 694356675654983680
Error for: 694352839993344000
Error for: 694342028726001664
Error for: 694329668942569472
Error for: 694206574471057408
Error for: 694183373896572928
Error for: 694001791655137281
Error for: 693993230313091072
Error for: 693942351086120961
Error for: 693647888581312512
Error for: 693644216740769793
Error for: 693642232151285760
Error for: 693629975228977152
Error for:

Error for: 680176173301628928
Error for: 680161097740095489
Error for: 680145970311643136
Error for: 680130881361686529
Error for: 680115823365742593
Error for: 680100725817409536
Error for: 680085611152338944
Error for: 680070545539371008
Error for: 680055455951884288
Error for: 679877062409191424
Error for: 679872969355714560
Error for: 679862121895714818
Error for: 679854723806179328
Error for: 679844490799091713
Error for: 679828447187857408
Error for: 679777920601223168
Error for: 679736210798047232
Error for: 679729593985699840
Error for: 679722016581222400
Error for: 679530280114372609
Error for: 679527802031484928
Error for: 679511351870550016
Error for: 679503373272485890
Error for: 679475951516934144
Error for: 679462823135686656
Error for: 679405845277462528
Error for: 679158373988876288
Error for: 679148763231985668
Error for: 679132435750195208
Error for: 679111216690831360
Error for: 679062614270468097
Error for: 679047485189439488
Error for: 679001094530465792
Error for:

Rate limit reached. Sleeping for: 703


Error for: 677301033169788928
